<a href="https://colab.research.google.com/github/rpdahxn/GANsInAction/blob/main/ch06_ProGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 6.2.1 고해상도 층으로 점진적 증대와 단계적 도입

In [1]:
import tensorflow as tf
import tensorflow.keras as K

In [3]:
def upscale_layer(layer, upscale_factor):
  # upscale_factor만큼 층(텐서)를 업스케일
  # 텐서 크기는 [group, height, width, channels]
  height, width = layer.get_shape()[1:3]
  size = (upscale_factor * height, upscale_factor * width)
  upscaled_layer = tf.image.resize_nearest_neighbor(layer, size)
  return upscaled_layer

In [4]:
def smoothly_merge_last_layer(list_of_layers, alpha):
  # 임곗값 알파를 기반으로 층을 부드럽게 합친다.
  # 이 함수는 모든 층이 이미 RGB로 바뀌었다고 가정한다.
  # 생성자를 위한 함수다.
  # list_of_layers : 해상도(크기) 순서대로 정렬된 텐서 리스트
  # alpha : (0, 1) 사이의 실수

  # 업스케일링을 위해 끝에서 두 번째 층을 선택
  last_fully_trained_layer = list_of_layers[-2]
  # 마지막으로 훈련된 층을 업스케일링
  last_layer_upscaled = upscale_layer(last_fully_trained_layer, 2)

  # 새로 추가된 층은 아직 완전히 훈련되지 않았다.
  larger_native_layer = list_of_layers[-1]

  # 합치기 전에 층 크기가 같은지 확인
  assert larger_native_layer.get_shape() == last_layer_upscaled.get_shape()

  # 곱셈은 브로드캐스팅되어 수행된다.
  new_layer = (1 - alpha) * last_layer_upscaled + larger_native_layer * alpha

  return new_layer 

### 6.2.3 미니배치 표준편차  
배치에 있는 샘플의 분산이 얼마인지 판별자에게 알려주어 모드 붕괴를 피한다.

In [ ]:
def minibatch_std_layer(layer, group_size=4):
    '''
    층의 미니배치 표준편차를 계산합니다.
    층의 데이터 타입은 float32로 가정합니다. 그렇지 않으면 타입 변환이 필요합니다.
    '''
    # 미니배치는 group_size로 나눌 수 있거나 group_size 보다 같거나 작아야 합니다.
    group_size = K.backend.minimum(group_size, tf.shape(layer)[0])

    # 간단하게 쓰기 위해 크기 정보를 따로 저장합니다. 
    # 그래프 실행 전에는 일반적으로 배치 차원이 None이기 때문에 tf.shape에서 이 크기를 얻습니다.
    shape = list(K.int_shape(input))
    shape[0] = tf.shape(input)[0]

    # 미니배치 수준에서 연산하기 위해 크기를 바꿉니다. 
    # 이 코드는 층이 [그룹(G), 미니배치(M), 너비(W), 높이(H), 채널(C)]라고 가정합니다. 
    # 하지만 씨아노(Theano) 방식의 순서를 사용하는 구현도 있습니다.
    minibatch = K.backend.reshape(layer, (group_size, -1, shape[1], shape[2], shape[3]))

    # [M, W, H, C] 그룹의 평균을 계산합니다.
    minibatch -= tf.reduce_mean(minibatch, axis=0, keepdims=True)
    # [M, W, H, C] 그룹의 분산을 계산합니다.
    minibatch = tf.reduce_mean(K.backend.square(minibatch), axis = 0)
    # [M,W,H,C] 그룹의 표준편차를 계산합니다.
    minibatch = K.backend.square(minibatch + 1e-8)
    # 특성 맵을 평균하여 [M,1,1,1] 픽셀을 얻습니다.
    minibatch = tf.reduce_mean(minibatch, axis=[1,2,3], keepdims=True)
    # 스칼라 값을 그룹과 픽셀에 맞게 변환합니다.
    minibatch = K.backend.tile(minibatch, [group_size, 1, shape[2], shape[3]])
    # 새로운 특성 맵을 추가합니다.
    return K.backend.concatenate([layer, minibatch], axis=1)